# Part 1

In [1]:
import re

step_re = re.compile(r'^Step (\w) must be finished before step (\w) can begin.$')

def parse_step(text):
    ''' Parse a line and return a tuple. '''
    match = step_re.match(text)
    if not match:
        raise Exception('Cannot parse step: {}'.format(text))
    return match.group(1,2)

In [2]:
parse_step('Step T must be finished before step X can begin.')

('T', 'X')

In [3]:
test_text = '''Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.'''

In [4]:
import collections

def parse_steps(text):
    '''
    Build up a tree from a list of instructions.
    
    The tree is a dictionary where the key is a step name
    and the value is a list of steps that depend on that step.
    i.e. {'A':['B','C']} means that A is the root and has edges pointing at B
    and C.
    '''
    steps = collections.defaultdict(list)
    for line in text.split('\n'):
        parent, child = parse_step(line)
        steps[parent].append(child)
    return steps

In [5]:
test_steps = parse_steps(test_text)
print(test_steps)

defaultdict(<class 'list'>, {'C': ['A', 'F'], 'A': ['B', 'D'], 'B': ['E'], 'D': ['E'], 'F': ['E']})


In [6]:
def find_roots(steps):
    ''' For a given list of steps, find the root nodes, i.e. the ones with 
    zero inbound edges. '''
    parent_nodes = set(steps.keys())
    child_nodes = set()
    for parent, children in steps.items():
        child_nodes.update(children)
    return parent_nodes - child_nodes

In [7]:
print(find_roots(test_steps))

{'C'}


In [8]:
def order_steps(steps):
    ''' Starting from root, traverse the tree and return a list of nodes. '''
    # Construct tree with reversed edges:
    backlinks = collections.defaultdict(set)
    for parent, children in steps.items():
        for child in children:
            backlinks[child].add(parent)
    roots = find_roots(steps)
    for root in roots:
        backlinks[root] = set()
    # The problem roughly describes the algorithm to use, so I've tried to translate
    # that prose into code.
    available = roots
    visited = set()
    ordered = list()
    while available:
        # Find a node who's dependencies are satisfied:
        for node in sorted(available):
            if backlinks[node].issubset(visited):
                ordered.append(node)
                visited.add(node)
                available.remove(node)
                available.update(steps[node])
                break
        else:
            raise Exception('Did not find a suitable node')
    return ''.join(ordered)

In [9]:
print(order_steps(test_steps))

CABDFE


In [10]:
with open('input.txt') as input_:
    steps = parse_steps(input_.read().strip())

In [11]:
print(order_steps(steps))

GKPTSLUXBIJMNCADFOVHEWYQRZ


# Part 2

In [12]:
import itertools

def time_steps(steps, n_workers, step_time=60, debug=False):
    ''' Starting from root, traverse the tree and return a list of nodes. '''
    # Construct tree with reversed edges:
    backlinks = collections.defaultdict(set)
    for parent, children in steps.items():
        for child in children:
            backlinks[child].add(parent)
    roots = find_roots(steps)
    for root in roots:
        backlinks[root] = set()
    # The problem roughly describes the algorithm to use, so I've tried to translate
    # that prose into code.
    available = roots
    visited = set()
    ordered = list()
    worker_tasks = [None] * n_workers
    
    def next_task():
        for task in sorted(available):
            if backlinks[task].issubset(visited):
                available.remove(task)
                return task
        return None
    
    for t in itertools.count():
        # Assign steps to idle workers
        for i in range(len(worker_tasks)):
            if worker_tasks[i] is None:
                task = next_task()
                if task is None:
                    break
                worker_tasks[i] = task, ord(task) - 64 + step_time
        
        # For debugging, print the same table as shown in the problem:
        if debug:
            task_debug = ' '.join('{}-{:02}'.format(*t) if t else '   .' for t in worker_tasks)
            print('{:02} {} {}'.format(t, task_debug, ''.join(ordered)))
        
        # Decrement each task's remaining time
        for i in range(len(worker_tasks)):
            if worker_tasks[i] is not None:
                worker_tasks[i] = worker_tasks[i][0], worker_tasks[i][1] - 1
                if worker_tasks[i][1] == 0:
                    name = worker_tasks[i][0]
                    ordered.append(name)
                    visited.add(name)
                    available.update(steps[name])
                    worker_tasks[i] = None
        
        # End when all tasks are complete
        if not available and not any(worker_tasks):
            break
    return t + 1

In [13]:
time_steps(test_steps, 2, step_time=0, debug=True)

00 C-03    . 
01 C-02    . 
02 C-01    . 
03 A-01 F-06 C
04 B-02 F-05 CA
05 B-01 F-04 CA
06 D-04 F-03 CAB
07 D-03 F-02 CAB
08 D-02 F-01 CAB
09 D-01    . CABF
10 E-05    . CABFD
11 E-04    . CABFD
12 E-03    . CABFD
13 E-02    . CABFD
14 E-01    . CABFD


15

In [14]:
time_steps(steps, 5)

920